In [197]:
import pandas as pd
import numpy as np
import os
import datetime
import re

##### 数据清洗代码

In [198]:
#data source

url1 = "https://github.com/msmx2022/Economics/raw/main/JP-Telecom/FinanceRepo/NTT/1-data/fy2021q4hosoku0512.xlsx"
url2 = "https://github.com/msmx2022/Economics/raw/main/JP-Telecom/FinanceRepo/NTT/1-data/fy2023q4hosoku0510.xlsx"

In [199]:
# Function to forward fill missing values (with special handling for first row)
def forward_fill(df, cols):
    for col in cols:
        df[col] = df[col].fillna(method='ffill')
    return df




In [200]:
#自定义数据列与数据所在地

#数据列
NTTCol1 = ["Category0","Category1","Category2","Category3","Empty","Empty","Unit"]
NTTCol2 = ["FY20Q1","FY20Q2","FY20Q3","Empty","FY20Q4","Empty"]
NTTCol3 = ["FY21Q1","FY21Q2","FY21Q3","Empty","FY21Q4","Empty","FY22Est"]
NTTCol = NTTCol1+NTTCol2+NTTCol3

#数据所在地
sheetname1 ='11(J)'

In [201]:
#import data
dfA = pd.read_excel(url1, sheet_name=sheetname1,engine='openpyxl')
                   
#rename columns
dfA.columns=NTTCol

#drop null rows in Unit
dfA = dfA[dfA['Unit'].notna()].reset_index(drop=True)

# Apply forward fill to DataFrame
dfA2 = forward_fill(dfA.copy(), ["Category0"])

dfA2a = dfA2[dfA2['Category0'].str.startswith('フレッツ光（コラボ光含む）')]
dfA2b = dfA2[~dfA2['Category0'].str.startswith('フレッツ光（コラボ光含む）')]

dfA2a1 = forward_fill(dfA2a.copy(), ["Category1"])

dfA3 =  pd.concat([dfA2a1,dfA2b]).reset_index(drop=True)
dfA4 = dfA3.drop(columns=["Empty"])

dfA4['Category1'] = dfA4['Category1'].fillna('NTT東西/NTT East&West')  # Replace NaN with 'Unknown'
dfA4a = dfA4[dfA4['Category1'].str.startswith('（再掲）')]
dfA4b = dfA4[~dfA4['Category1'].str.startswith('（再掲）')].drop(columns=['Category2','Category3'])

# repalce（注*）的模式
dfA4b['Category0'] = dfA4b['Category0'].str.replace(r'（注\d+）', '', regex=True)

dfA4b=dfA4b.rename(columns={'Category1':'Category2'})
dfA4b=dfA4b.rename(columns={'Category0':'Category1'})

#删除Mobile数据
categories_to_remove = ["携帯電話サービス解約率","携帯電話サービス純増数","携帯電話サービス契約数", "iモード", "spモード等","総販売数","スマートフォン・タブレット販売数"]
dfA4b = dfA4b[~dfA4b['Category1'].isin(categories_to_remove)]


# 定义日文注释与英文注释的对应关系
translation_map1 = {
    "フレッツ光（コラボ光含む）":"フレッツ光（コラボ光含む）/ FLET'S Hikari（with Collaboration）",
    "加入電話": "加入電話 / Telephone",
    "INSネット": "INSネット / INS-Net",
    "加入電話＋INSネット": "加入電話＋INSネット / Telephone + INS-Net",
    "公衆電話": "公衆電話 / Public telephones",
    "フレッツ・ISDN": "フレッツ・ISDN / FLET'S ISDN",
    "フレッツ・ADSL": "フレッツ・ADSL / FLET'S ADSL",
    "ひかり電話": "ひかり電話 / Hikari Tel",
    "フレッツ・テレビ伝送サービス": "フレッツ・テレビ伝送サービス / FLET'S TV"
}

dfA4b['Category1'] = dfA4b['Category1'].map(translation_map1)


translation_map2={
    'NTT東日本':'NTT東日本 / NTT East',
    'NTT西日本':'NTT西日本 / NTT West',
    'NTT東西/NTT East&West':'NTT東西/NTT East&West'
}


dfA4b['Category2'] = dfA4b['Category2'].map(translation_map2)



translation_map3={
    '（千契約）':'（千/thousands）',
    '（千チャネル）':'（千チャネル/thousands channel）',
    '（個）':'（個/number）'
}


dfA4b['Unit'] = dfA4b['Unit'].map(translation_map3)


#横縦変換
dfA5b=dfA4b.melt(id_vars=['Category1','Category2','Unit'],
               value_vars=dfA4b.columns[3:],
               var_name='Period',
               value_name='Value')

In [202]:
#自定义数据列与数据所在地

#数据列
NTTCol1 = ["Category0","Category1","Category2","Category3","Empty","Empty","Unit"]
NTTCol2 = ["FY22Q1","FY22Q2","FY22Q3","Empty","FY22Q4","Empty"]
NTTCol3 = ["FY23Q1","FY23Q2","FY23Q3","Empty","FY23Q4","Empty","FY24Est"]
NTTCol = NTTCol1+NTTCol2+NTTCol3

#数据所在地
sheetname2 ='13'

In [203]:
#import data
dfB = pd.read_excel(url2, sheet_name=sheetname2,engine='openpyxl')
                   
#rename columns
dfB.columns=NTTCol

#drop null rows in Unit
dfB = dfB[dfB['Unit'].notna()].reset_index(drop=True)


# Apply forward fill to the As-is DataFrame
dfB2 = forward_fill(dfB.copy(), ["Category1"])

dfB2a = dfB2[dfB2['Category1'].str.startswith('フレッツ光（コラボ光含む）')]
dfB2b = dfB2[~dfB2['Category1'].str.startswith('フレッツ光（コラボ光含む）')]

dfB2a1 = forward_fill(dfB2a.copy(), ["Category2"])


dfB3 =  pd.concat([dfB2a1,dfB2b]).reset_index(drop=True)
dfB4 = dfB3.drop(columns=["Empty"])

dfB4['Category2'] = dfB4['Category2'].fillna('NTT東西/NTT East&West')  # Replace NaN with 'Unknown'
dfB4a = dfB4[dfB4['Category2'].str.startswith('（再掲）')]
dfB4b = dfB4[~dfB4['Category2'].str.startswith('（再掲）')].drop(columns=['Category0','Category3'])

# repalce（注*）的模式
dfB4b['Category1'] = dfB4b['Category1'].str.replace(r'（\d+）', '', regex=True)

# 定义日文注释与英文注释的对应关系
translation_map = {
    "フレッツ光（コラボ光含む）/ FLET'S Hikari（including Hikari Collaboration Model）":"フレッツ光（コラボ光含む）/ FLET'S Hikari（with Collaboration）",
    "加入電話 / Telephone subscriber lines": "加入電話 / Telephone",
    "INSネット / INS-Net": "INSネット / INS-Net",
    "加入電話＋INSネット / Telephone subscriber lines + INS-Net": "加入電話＋INSネット / Telephone + INS-Net",
    "公衆電話 / Public telephones": "公衆電話 / Public telephones",
    "フレッツ・ISDN / FLET'S ISDN": "フレッツ・ISDN / FLET'S ISDN",
    "フレッツ・ADSL / FLET'S ADSL": "フレッツ・ADSL / FLET'S ADSL",
    "ひかり電話 / Hikari Denwa": "ひかり電話 / Hikari Tel",
    "フレッツ・テレビ伝送サービス / FLET'S TV Transmission Services": "フレッツ・テレビ伝送サービス / FLET'S TV"
}

dfB4b['Category1'] = dfB4b['Category1'].map(translation_map)



#横縦変換
dfB5b=dfB4b.melt(id_vars=['Category1','Category2','Unit'],
               value_vars=dfB4b.columns[3:],
               var_name='Period',
               value_name='Value')

c:\Users\A25563592\AppData\Local\Programs\Python\Python310\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [204]:
df6 =  pd.concat([dfA5b,dfB5b]).reset_index(drop=True)


#modify Unit
df6a = df6[~(df6['Category1']=='公衆電話 / Public telephones')].reset_index(drop=True)
df6b = df6[(df6['Category1']=='公衆電話 / Public telephones')].reset_index(drop=True)


df6b['Value'] = df6b['Value']/1000

translation_map={
    '（千/thousands）':'（千/thousands）',
    '（千チャネル/thousands channel）':'（千チャネル/thousands channel）',
    '（個/number）':'（千/thousands）'
}

df6b['Unit'] = df6b['Unit'].map(translation_map)


df7 =  pd.concat([df6a,df6b]).reset_index(drop=True)

In [205]:
#export
df7.to_excel('XXXXXXX\\DataAnalysis\\02_Viz\\Carrier\\3-output\\NTTEastWestSubscription.xlsx', index=False)